In [151]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import seaborn as sns
from sklearn.model_selection import train_test_split


In [12]:
import statsmodels.api as sm
import statsmodels.tools

In [152]:
df = pd.read_csv("Life Expectancy Data.csv")

In [5]:
df.dtypes

Country                         object
Region                          object
Year                             int64
Infant_deaths                  float64
Under_five_deaths              float64
Adult_mortality                float64
Alcohol_consumption            float64
Hepatitis_B                      int64
Measles                          int64
BMI                            float64
Polio                            int64
Diphtheria                       int64
Incidents_HIV                  float64
GDP_per_capita                   int64
Population_mln                 float64
Thinness_ten_nineteen_years    float64
Thinness_five_nine_years       float64
Schooling                      float64
Economy_status_Developed         int64
Economy_status_Developing        int64
Life_expectancy                float64
dtype: object

In [153]:
fcols = list(df.columns)
fcols.remove("Life_expectancy")
fcols.remove("Country")

In [154]:
X = df[fcols]
y = df.Life_expectancy

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [156]:
def feature_engineering(df):
    df = df.copy()
    df = pd.get_dummies(df, columns=["Region"], drop_first=True, prefix="region", dtype=int)
    df["GDP_per_capita"] = df["GDP_per_capita"] / 1000
    df["gdp_per_capita_exp"] = np.exp(df["GDP_per_capita"], dtype=np.float64)
    df["alcohol_exp"] = np.exp(df["Alcohol_consumption"])
    df = sm.add_constant(df)
    return df

In [76]:
X_train_fe = feature_engineering(X_train)

In [157]:
feature_cols = ['const', 'Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'alcohol_exp']


stepwise_recommended_cols = ['Infant_deaths', 'Adult_mortality', 'Economy_status_Developed', 'region_Central America and Caribbean', 'region_South America', 'Under_five_deaths', 'GDP_per_capita', 'region_Oceania', 'region_European Union', 'Schooling', 'BMI', 'Year', 'Incidents_HIV', 'Hepatitis_B']

In [78]:
lin_reg = sm.OLS(y_train, X_train_fe[feature_cols])

In [79]:
results = lin_reg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     5365.
Date:                Tue, 28 May 2024   Prob (F-statistic):               0.00
Time:                        10:41:07   Log-Likelihood:                -3650.9
No. Observations:                2291   AIC:                             7356.
Df Residuals:                    2264   BIC:                             7511.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                    8.7503      7.663      1.142      0.254      -6.277      23.777
Year                                     0.0357      0.006      6.231      0.000       0.024       0.047
Infant_deaths                           -0.0521      0.006     -8.198      0.000      -0.065      -0.040
Under_five_deaths                       -0.0510      0.004    -12.674      0.000      -0.059      -0.043
Adult_mortality                         -0.0468      0.001    -75.767      0.000      -0.048      -0.046
Alcohol_consumption                      0.0051      0.012      0.433      0.665      -0.018       0.028
Hepatitis_B                             -0.0075      0.003     -2.912      0.004      -0.013      -0.002
Measles                                  0.0028      0.002      1.578      0.115      -0.001       0.006
BMI                                     -0.1347      0.023     -5.946      0.000      -0.179      -0.090
Polio                                    0.0040      0.006      0.672      0.501      -0.008       0.016
Diphtheria                              -0.0041      0.006     -0.685      0.493      -0.016       0.008
Incidents_HIV                            0.0854      0.018      4.807      0.000       0.051       0.120
GDP_per_capita                           0.0190      0.002      7.955      0.000       0.014       0.024
Population_mln                          -0.0003      0.000     -1.348      0.178      -0.001       0.000
Thinness_ten_nineteen_years             -0.0340      0.018     -1.878      0.061      -0.070       0.002
Thinness_five_nine_years                 0.0244      0.018      1.348      0.178      -0.011       0.060
Schooling                                0.1059      0.018      5.724      0.000       0.070       0.142
Economy_status_Developed                 5.6121      3.828      1.466      0.143      -1.895      13.119
Economy_status_Developing                3.1382      3.836      0.818      0.413      -4.385      10.662
region_Asia                              0.3416      0.113      3.024      0.003       0.120       0.563
region_Central America and Caribbean     1.8946      0.121     15.687      0.000       1.658       2.131
region_European Union                   -0.7429      0.178     -4.185      0.000      -1.091      -0.395
region_Middle East                       0.2110      0.139      1.521      0.128      -0.061       0.483
region_North America                     0.6628      0.242      2.736      0.006       0.188       1.138
region_Oceania                          -0.7781      0.147     -5.288      0.000      -1.067      -0.490
region_Rest of Europe                    0.1786      0.141      1.263      0.207      -0.099       0.456
region_South America                  

In [80]:
y_pred = results.predict(X_train_fe[feature_cols])
rmse = statsmodels.tools.eval_measures.meanabs(y_train, y_pred)
print(rmse)

0.9490168521941291


In [81]:
X_test_fe = feature_engineering(X_test)
X_test_fe = X_test_fe[feature_cols]
y_test_pred = results.predict(X_test_fe)
rmse = statsmodels.tools.eval_measures.meanabs(y_test, y_test_pred)
print(rmse)

0.940932582628842


In [82]:
def stepwise_selection(X, y, threshold_in = 0.01, threshold_out = 0.05, verbose = True):
    # The function is checking for p-values (whether features are statistically significant) - lower is better
    included = [] # this is going to be the list of features we keep
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index = excluded, dtype = 'float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        # we add the feature with the lowest (best) p-value under the threshold to our 'included' list
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval)) # specifying the verbose text


        # backward step: removing features if new features added to the list make them statistically insignificant
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        # if the p-value exceeds the upper threshold, the feature will be dropped from the 'included' list
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [129]:
result = stepwise_selection(X_train_fe[cols_ethical], y_train)

print('resulting features:')
print(result)
stepwise_recommended_cols = result

Add  Schooling                      with p-value 0.0
Add  const                          with p-value 0.0
Add  GDP_per_capita                 with p-value 5.57902e-42
Add  region_Central America and Caribbean with p-value 4.21017e-24
Add  region_Middle East             with p-value 2.08459e-25
Add  region_South America           with p-value 1.86518e-27
Add  region_Asia                    with p-value 2.4012e-28
Add  Economy_status_Developed       with p-value 3.5445e-38
Add  Economy_status_Developing      with p-value 0.0
Add  region_Rest of Europe          with p-value 8.50019e-18
Add  region_Oceania                 with p-value 2.79044e-23
Add  Year                           with p-value 1.90938e-24
Add  region_European Union          with p-value 6.3627e-25
Add  region_North America           with p-value 2.58771e-29
Add  Alcohol_consumption            with p-value 3.08702e-19
Add  Population_mln                 with p-value 0.00818192
resulting features:
['Schooling', 'const', 'GD

In [49]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

cols = ['Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'alcohol_exp']

pd.Series([variance_inflation_factor(X_train_fe[cols].values, i) for i in range(X_train_fe[cols].shape[1])], index = X_train_fe[cols].columns)

Year                                         1.136409
Infant_deaths                               49.933677
Under_five_deaths                           51.856138
Adult_mortality                              8.095150
Alcohol_consumption                          3.458269
Hepatitis_B                                  2.766374
Measles                                      1.693158
BMI                                          3.916793
Polio                                       12.952250
Diphtheria                                  13.918846
Incidents_HIV                                3.045644
GDP_per_capita                               2.592829
Population_mln                               1.214619
Thinness_ten_nineteen_years                 10.282747
Thinness_five_nine_years                    10.782697
Schooling                                    5.490648
Economy_status_Developed                 43245.305913
Economy_status_Developing               167795.691169
region_Asia                 

In [145]:
def calculate_vif(X, thresh = 5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        # this bit uses list comprehension to gather all the VIF values of the different variables
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]
        
        maxloc = vif.index(max(vif)) # getting the index of the highest VIF value
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc] # we delete the highest VIF value on condition that it's higher than the threshold
            dropped = True # if we deleted anything, we set the 'dropped' value to True to stay in the while loop

    print('Remaining variables:')
    print(X.columns[variables]) # finally, we print the variables that are still in our set
    return X.iloc[:, variables] # and return our X cut down to the remaining variables


calculate_vif(X_train_fe[cols_ethical_stepwise])

c:\Users\matth\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\matth\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


dropping 'Economy_status_Developed' at index: 7
dropping 'const' at index: 1
dropping 'Year' at index: 9
dropping 'Schooling' at index: 0
dropping 'Alcohol_consumption' at index: 10
Remaining variables:
Index(['GDP_per_capita', 'region_Central America and Caribbean',
       'region_Middle East', 'region_South America', 'region_Asia',
       'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania',
       'region_European Union', 'region_North America', 'Population_mln'],
      dtype='object')


,GDP_per_capita,region_Central America and Caribbean,region_Middle East,region_South America,region_Asia,Economy_status_Developing,region_Rest of Europe,region_Oceania,region_European Union,region_North America,Population_mln
2291,58.422,0,1,0,0,1,0,0,0,0,3.30
1188,27.026,0,0,0,0,0,0,0,1,0,45.95
772,1.087,1,0,0,0,1,0,0,0,0,0.15
1336,19.068,0,0,0,0,0,0,0,1,0,10.42
429,10.859,0,0,0,0,0,0,0,1,0,38.04
...,...,...,...,...,...,...,...,...,...,...,...
2763,1.037,0,1,0,0,1,0,0,0,0,19.58
905,0.306,0,0,0,0,1,0,0,0,0,8.13
1096,0.471,0,0,0,0,1,0,0,0,0,18.34
235,12.695,1,0,0,0,1,0,0,0,0,3.84


In [2]:
def LinReg(cols):
    X_train_fe = feature_engineering(X_train)
    lin_reg = sm.OLS(y_train, X_train_fe[cols])
    results = lin_reg.fit()
    y_pred = results.predict(X_train_fe[cols])
    train_rmse = statsmodels.tools.eval_measures.meanabs(y_train, y_pred)
    print(f"Training data RMSE:\t{train_rmse}")
    X_test_fe = feature_engineering(X_test)
    X_test_fe = X_test_fe[cols]
    y_test_pred = results.predict(X_test_fe)
    test_rmse = statsmodels.tools.eval_measures.meanabs(y_test, y_test_pred)
    print(f"Testing data RMSE:\t{test_rmse}")
    return results

results = LinReg(feature_cols)
results.summary()

NameError: name 'feature_cols' is not defined

In [1]:
vif_cols = ['Under_five_deaths', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'alcohol_exp', 'const']

step_cols = ['Adult_mortality', 'Economy_status_Developed', 'region_Central America and Caribbean', 'region_South America', 'Under_five_deaths', 'GDP_per_capita', 'region_Oceania','region_European Union', 'Schooling', 'BMI', 'Year', 'Incidents_HIV', 'Hepatitis_B', 'const']

step_cols_vif = ['region_Central America and Caribbean', 'region_South America',
       'Under_five_deaths', 'GDP_per_capita', 'region_Oceania',
       'region_European Union', 'Schooling', 'Incidents_HIV', 'const']

cols = ['const', 'Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'gdp_per_capita_exp',
       'alcohol_exp']

cols_ethical = ['const', 'Year', 'Alcohol_consumption', 'GDP_per_capita',
       'Population_mln', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America']

cols_ethical_stepwise = ['Schooling', 'const', 'GDP_per_capita', 'region_Central America and Caribbean', 'region_Middle East', 'region_South America', 'region_Asia', 'Economy_status_Developed', 'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania', 'Year', 'region_European Union', 'region_North America', 'Alcohol_consumption', 'Population_mln']

cols_ethical_vif = ['GDP_per_capita', 'Population_mln', 'Economy_status_Developing',
       'region_Asia', 'region_Central America and Caribbean',
       'region_European Union', 'region_Middle East', 'region_North America',
       'region_Oceania', 'region_Rest of Europe', 'region_South America', 'const']

cols_ethical_stepwise_vif = ['GDP_per_capita', 'region_Central America and Caribbean',
       'region_Middle East', 'region_South America', 'region_Asia',
       'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania',
       'region_European Union', 'region_North America', 'Population_mln']

cols_sumye = ['const',
       'Under_five_deaths', 
       'Adult_mortality',
       'GDP_per_capita_thousand',
        'Population_mln'
               ]

LinReg(cols_sumye).summary()

NameError: name 'LinReg' is not defined